In [24]:
import mlflow
import pandas as pd
from langchain.llms import GPT4All
import pinecone
from langchain.embeddings import GPT4AllEmbeddings
import os
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
from langchain import hub
from src.constants import EVAL_QUESTIONS

In [25]:
load_dotenv()

True

In [26]:
pinecone.init(
    api_key = os.getenv("PINECONE_API_KEY"),
    environment = os.getenv("PINECONE_ENV"),
)

In [27]:
index = pinecone.Index("clementine-loka")

In [28]:
rag_prompt = hub.pull("rlm/rag-prompt")
# llm = GPT4All(model="models/orca-mini-3b-gguf2-q4_0.gguf")
llm = GPT4All(model="models/gpt4all-falcon-q4_0.gguf")

In [29]:
def load_retriever(persist_directory):
    embeddings = GPT4AllEmbeddings()
    vectorstore = Pinecone.from_existing_index("clementine-loka", embeddings)
    retriever = vectorstore.as_retriever()
    return retriever

In [30]:
embeddings = GPT4AllEmbeddings()
vectorstore = Pinecone.from_existing_index("clementine-loka", embeddings)

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [31]:
vectorstore.similarity_search("SageMaker Geospatial capabilities")[0].page_content#metadata["source"]

'SageMaker geospatial capabilities roles\n\nAs a managed service, Amazon SageMaker geospatial capabilities perform operations on your behalf on the AWS hardware that is managed by SageMaker. It can perform only operations that the user permits.\n\nA user can grant these permissions with an IAM role (referred to as an execution role).\n\nTo create and use a locally available execution role, you can use the following procedures.\n\nCreate an execution role\n\nTo work with SageMaker geospatial capabilities you need to setup a user role and an execution role. A user role is an AWS identity with permission policies that determine what the user can and can not do within AWS. An execution role is an IAM role that grants the service permission to access your AWS resources. An execution role consists of permissions and trust policy. The trust policy specifies which principals have the permission to assume the role.'

In [32]:
ANSWERS = [
    "Amazon SageMaker is a fully managed service that provides every developer and data scientist with the ability to build, train, and deploy machine learning (ML) models",
    "All supported AWS regions except China (Beijing), Asia Pacific (Jakarta), Middle East (UAE), Asia Pacific (Hyderabad), Asia Pacific (Melbourne), AWS GovCloud (US-East), AWS GovCloud (US-West), Europe (Spain), China (Ningxia), Europe (Zurich) Region",
    "Checks whether AWS Key Management Service (KMS) key is configured for an Amazon SageMaker endpoint configuration. The rule is NON_COMPLIANT if 'KmsKeyId' is not specified for the Amazon SageMaker endpoint configuration.",
    "SageMaker geospatial capabilities rolesAs a managed service, Amazon SageMaker geospatial capabilities perform operations on your behalf on the AWS hardware that is managed by SageMaker. It can perform only operations that the user permits."
]

In [33]:
retriever=load_retriever("")
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type='stuff',
    verbose=True,
    retriever=retriever,
    chain_type_kwargs={"prompt": rag_prompt},
    return_source_documents=False
)

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [34]:
question = "What are all AWS regions where SageMaker is available?"
result = qa_chain({"query": question})
result["result"]



> Entering new RetrievalQA chain...

> Finished chain.


' The answer is "sagemaker-notebook-instance-root-access-check".'

In [35]:
model_name = "gpt4all-falcon"
persist_dir="database"
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))

with mlflow.start_run(run_name="log_model_"+ model_name):
    mlflow.log_param("model_name", model_name)
    
    logged_model = mlflow.langchain.log_model(
        qa_chain, 
        artifact_path="model",
        loader_fn=load_retriever,
        persist_dir=persist_dir,
    )
    
    questions = pd.DataFrame({"query": EVAL_QUESTIONS,
                              "answer": ANSWERS})
    generated = mlflow.evaluate(
        model=logged_model.model_uri,
        # model_type="question-answering",
        data=questions,
        targets="answer",
        extra_metrics=[
            mlflow.metrics.toxicity(), 
            mlflow.metrics.latency(), 
            mlflow.metrics.ari_grade_level(), 
            mlflow.metrics.flesch_kincaid_grade_level(),
            mlflow.metrics.exact_match(),
        ],
    )
    mlflow.end_run()

2023/12/07 17:11:12 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/cg/6bd366p92fbb82w4cdb91yd40000gn/T/tmpq6fh6yqv/model, flavor: langchain), fall back to return ['langchain==0.0.346']. Set logging level to DEBUG to see the full traceback.
/Users/uribe/.local/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/uribe/.local/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


/Users/uribe/miniconda3/lib/python3.11/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
/Users/uribe/miniconda3/lib/python3.11/site-packages/mlflow/models/evaluation/base.py:414: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(_hash_array_like_element_as_bytes)
2023/12/07 17:11:24 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/12/07 17:11:24 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.




> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...


2023/12/07 17:12:29 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...



> Finished chain.


2023/12/07 17:12:30 INFO mlflow.models.evaluation.default_evaluator: Evaluating metrics: toxicity
2023/12/07 17:12:30 INFO mlflow.models.evaluation.default_evaluator: Evaluating metrics: ari_grade_level
2023/12/07 17:12:30 INFO mlflow.models.evaluation.default_evaluator: Evaluating metrics: flesch_kincaid_grade_level
2023/12/07 17:12:30 INFO mlflow.models.evaluation.default_evaluator: Evaluating metrics: exact_match
